<a href="https://colab.research.google.com/github/gc2321/apache_beam/blob/main/2_combine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import files
# uploaded = files.upload()

Saving dept_data.txt to dept_data.txt


In [ ]:
#!{'pip install --quiet apache_beam'}

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.0/670.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup

In [ ]:
#!{'pip install --quiet apache-beam[interactive]'}

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 406.3/406.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 816.5/816.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 21.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipykernel==5.5.6, but you have ipykernel 6.29.4 which is incompatible.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 8.24.0 which is incompatible.


In [ ]:
import apache_beam as beam

In [ ]:
# Mount Google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import fileinput
filePath ="/content/gdrive/MyDrive/beam/dept_data.txt"

In [ ]:
with beam.Pipeline() as p1:
  attendance_count = (
      p1
      | 'Read from file' >> beam.io.ReadFromText(filePath)
      | 'Split Lines of file' >> beam.Map(lambda record: record.split(','))
      | 'Filter by Dept' >> beam.Filter(lambda record: record[3] == 'Accounts')
      | 'Create Name : Count' >> beam.Map(lambda record: (record[1], 1))
      | 'CombinePerKey' >> beam.CombinePerKey(sum)
      | 'Format result' >> beam.Map(lambda employee_count: str(employee_count))
      | 'Output' >> beam.io.WriteToText('data/output')
  )

In [ ]:
# sample the first 20 results
!head -n 20 data/output-00000-of-00001

('Marco', 31)
('Rebekah', 31)
('Itoe', 31)
('Edouard', 31)
('Kyle', 62)
('Kumiko', 31)
('Gaston', 31)
('Ayumi', 30)


## Branching Pipeline

In [ ]:
# ParDo
class SplitRow(beam.DoFn):
  def process(self, element):
    return [element.split(',')]

class FilterAccountsEmployee(beam.DoFn):
  def __init__(self, acc):
        self.acc = acc

  def process(self, element):
    if element[3] == self.acc:
      return [element]

class PairEmployee(beam.DoFn):
  def process(self, element):
    return [(element[3] + "," + element[1], 1)]

class Counting(beam.DoFn):
  def process(self, element):
    (key, values) = element
    return [(key, sum(values))]

In [ ]:
p = beam.Pipeline()


input_collection = (
                      p
                      | "Read from text file" >> beam.io.ReadFromText(filePath)
                      | "Split rows" >> beam.ParDo(SplitRow())
                   )

accounts_count = (
                      input_collection
                      | 'Get all Accounts dept persons' >> beam.ParDo(FilterAccountsEmployee('Accounts'))
                      | 'Pair each accounts employee with 1' >> beam.ParDo(PairEmployee())
                      | 'GroupByKey Acc' >> beam.GroupByKey()
                      | 'Count Group Acc' >> beam.ParDo(Counting())
                    #  | 'Write results for account' >> beam.io.WriteToText('data/Account')
                 )

hr_count = (
                input_collection
                | 'Get all HR dept persons' >> beam.ParDo(FilterAccountsEmployee('HR'))
                | 'Pair each hr employee with 1' >> beam.ParDo(PairEmployee())
                | 'GroupByKey HR' >> beam.GroupByKey()
                | 'Count Group HR' >> beam.ParDo(Counting())
                #| 'Write results for hr' >> beam.io.WriteToText('data/HR')
           )

output =(
         (accounts_count,hr_count)
    | beam.Flatten()
    | beam.io.WriteToText('data/both')
)

In [ ]:
p.run()

In [ ]:
!head -n 20 data/both-00000-of-00001

('Accounts, Marco', 31)
('Accounts, Rebekah', 31)
('Accounts, Itoe', 31)
('Accounts, Edouard', 31)
('Accounts, Kyle', 62)
('Accounts, Kumiko', 31)
('Accounts, Gaston', 31)
('Accounts, Ayumi', 30)
('HR, Beryl', 62)
('HR, Olga', 31)
('HR, Leslie', 31)
('HR, Mindy', 31)
('HR, Vicky', 31)
('HR, Richard', 31)
('HR, Kirk', 31)
('HR, Kaori', 31)
('HR, Oscar', 31)
